In [ ]:
# Selenium 설치.
#!pip install selenium

In [ ]:
from selenium import webdriver
import bs4,re
import pandas as pd
import numpy as np
import os
import time

In [ ]:
os.chdir(r"C:\Users\Gram\Desktop\myPyCode\프로젝트1")

In [ ]:
read_compare = pd.read_csv("data_coffee.csv",encoding='cp949')

In [ ]:
def grep(reg_exp,my_text):
    result = []
    reg_exp_compiled = re.compile(reg_exp)
    for i in range(len(my_text)):
        if reg_exp_compiled.search(my_text[i]):
            result.append(my_text[i])
    return result

In [ ]:
driver1 = webdriver.Chrome("./chromedriver")
driver2 = webdriver.Chrome("./chromedriver")

In [ ]:
driver1.get("https://v4.map.naver.com")

In [ ]:
driver1.find_elements_by_css_selector("button.btn_close")[1].click()

In [ ]:
gu_list =['중랑구','성북구','강북구','도봉구','노원구','은평구']
gu_list=['강북구']
enter_gu='강북구'

In [ ]:
for enter_gu in gu_list:
    search_box = driver1.find_element_by_css_selector("input#search-input")
    search_box.send_keys(enter_gu+'커피전문점')
    
    # 검색버튼 누르기 // 검색버튼: button.spm
    search_button = driver1.find_element_by_css_selector("button.spm")
    search_button.click()
    
    # 1초의 지연시간을 줍니다. 
    time.sleep(1)
    
    page = 1
    cafe_name=[]        # 카페 이름
    cafe_menu=[]        # 카페 메뉴들 리스트
    cafe_price=[]       # 메뉴 가격들 리스트
    cafe_gu=[]          # 지역
    cafe_classify =[]
    
    while True : 
        time.sleep(2)
    
        html = driver1.page_source
        soup = bs4.BeautifulSoup(html, 'html.parser')
        
        html_2 = driver2.page_source
        soup_2 = bs4.BeautifulSoup(html_2, 'html.parser')
    
        page = page + 1
        cafe_list = soup.select('ul.lst_site > li')
        
            
            
        for data in cafe_list:
                
            detail_url = data.select_one('a.spm_sw_detail').attrs['href']            
            driver2.get('https://map.naver.com' + detail_url)
            time.sleep(2)    
            
            #### 카페 이름
            driver2.find_element_by_css_selector("#content > div").text
            name = driver2.find_element_by_class_name("name").text
            
            ######## 카페 프렌차이즈vs개인
            
            for i in read_compare['store_name']:
                if i in name:
                    classify = 'F'
                    break
                else :
                    classify = 'P'
            
            
            ### 카페 메뉴 리스트
            emprice=[]        # 가격들
            spname=[]         # 메뉴 이름들
            
            try:
                list_menu = driver2.find_elements_by_css_selector("div.list_menu_inner")
            
                for i in range(len(list_menu)-1):
                    # 세부 데이터 수집
                    spname.append(list_menu[i].find_element_by_css_selector("span.name").text)
                    emprice.append(list_menu[i].find_element_by_css_selector("em.price").text   )
            ### nan 값이     
            except:
                spname.append(np.nan)
                emprice.append(np.nan)
                
            if spname :
                test=False
                for i in range(len(spname)):
                    if spname[i] !=[]:      ## 메뉴 리스트가 있다면
                        if grep("아메리카노",spname):     # True
                            find_menu=grep("아메리카노",spname)
                            cafe_menu.append(find_menu[0])          # 메뉴 입력
                            
                            price_ind=spname.index(find_menu[0])
                            expr=emprice[price_ind]                 ## 정규 표현식
                            expr=expr.replace("원","")
                            expr=expr.replace(",","")
                            if grep('~',expr):
                                expr=expr.split('~')
                                expr=expr[0]
                            cafe_price.append(expr)   # 가격 입력
                            test= True
                        
                        elif grep("카페라떼",spname):     # True
                            find_menu=grep("카페라떼",spname)
                            cafe_menu.append(find_menu[0])          # 메뉴 입력
                            
                            price_ind=spname.index(find_menu[0])
                            expr=emprice[price_ind]                 ## 정규 표현식
                            expr=expr.replace("원","")
                            expr=expr.replace(",","")
                            if grep('~',expr):
                                expr=expr.split('~')
                                expr=expr[0]
                            cafe_price.append(expr)   # 가격 입력
                            test= True
                        
                        elif grep("블랙커피",spname):     # True
                            find_menu=grep("블랙커피",spname)
                            cafe_menu.append(find_menu[0])          # 메뉴 입력
                            
                            price_ind=spname.index(find_menu[0])
                            expr=emprice[price_ind]                 ## 정규 표현식
                            expr=expr.replace("원","")
                            expr=expr.replace(",","")
                            if grep('~',expr):
                                expr=expr.split('~')
                                expr=expr[0]
                            cafe_price.append(expr)   # 가격 입력
                            test= True

                        else :
                            
                            cafe_menu.append(spname[0])
                            expr=emprice[0]                 ## 정규 표현식
                            expr=expr.replace("원","")
                            expr=expr.replace(",","")
                            if grep('~',expr):
                                expr=expr.split('~')
                                expr=expr[0]
                            cafe_price.append(expr)   # 가격 입력
                            test= True

                    
                    if test :
                        cafe_name.append(name)
                        cafe_gu.append(enter_gu)
                        cafe_classify.append(classify)
                        break
            
            else:
                cafe_name.append(name)
                cafe_gu.append(enter_gu)
                cafe_menu.append(np.nan)
                cafe_price.append(np.nan)
                cafe_classify.append(classify)
                
                
    
            time.sleep(1)
    
            
        try:
            if page % 5 == 1:
                driver1.find_element_by_class_name('next').click()
            else:
                driver1.find_element_by_xpath('//a[text()=' + str(page) + ']').click()
    
        except:
            break

In [ ]:
#데이터프레임으로 저장
test={"카페":cafe_name,"메뉴":cafe_menu,"가격":cafe_price,"지역":cafe_gu,"구분":cafe_classify}
df_test=pd.DataFrame(test,columns=['카페','메뉴','가격','지역',"구분"])    

In [ ]:
#excel로 저장
df_test.to_excel(enter_gu+".xlsx")

In [ ]:
#test 불러오기
#read_Test = pd.read_excel("test.xlsx")